# Neighbourhood Segmentation and Clustering

This notebook is used to explore, segment and cluster neighbourboods in the city of Toronto.

## Part 1: Dataframe Creation

In [1]:
import pandas as pd
import requests

# !conda install -c anaconda beautifulsoup4
# Note: Commented above line since package was installed after first run and hence not required to be rerun
from bs4 import BeautifulSoup

In [2]:
# Parsing source html page
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html.parser')

In [3]:
# Importing parsed data into dataframe
table = soup.find_all('table', class_='wikitable sortable')
df = pd.read_html(str(table))[0] # Note: Included [0] to access item in list as function will always return a list of DataFrames
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
# Processing data in dataframe

# Course Instruction: The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

# Course Instruction: Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df['Borough'] != 'Not assigned']

# Course Instruction: More than one neighborhood can exist in one postal code area.
# For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.
# These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).to_frame().reset_index()

# Course Instruction: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']

df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
# Printing number of rows of dataframe

# Course Instruction: In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df.shape

(103, 3)

## Part 2: Coordinates Addition

In [6]:
df_coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
df_coordinates.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

df = df.merge(df_coordinates, on='PostalCode')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 3: Neighbourhood Clustering

### 3.1: Explore Neighbourhoods in Toronto

In [7]:
# Defining Foursquare Credentials

CLIENT_ID = 'XSOZCHLB5OT4IVJGPRWPTC3FWSA1LCCOXBPNLJ1O3ZI353LD'
CLIENT_SECRET = 'HJY0IEW2HTTM4HKFYDIA1I0ID3JNWR2CFZJJCSJG0JT5IBJC'
VERSION = '20180605'

In [8]:
# Defining function to get top venues of all neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius, limit):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
# Running function on each neighbourhood and creating a new dataframe called toronto_venues

toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                 latitudes=df['Latitude'],
                                 longitudes=df['Longitude'],
                                 radius=500,
                                 limit=100
                                )

toronto_venues.head()

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence P

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [10]:
# Checking size of resulting dataframe

toronto_venues.shape

(2115, 7)

In [11]:
# Checking number of unique categories curated from all returned venues

print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 266 unique categories.


### 3.2: Analyse Each Neighbourhood

In [12]:
# One hot encoding for top venues of all neighbourhoods in Toronto

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['TORNeighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,TORNeighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# Checking size of resulting dataframe

toronto_onehot.shape

(2115, 267)

In [14]:
# Grouping rows by neighbourhood and calculating mean of frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby(['TORNeighborhood']).mean().reset_index()
toronto_grouped.head()

,TORNeighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Checking size of resulting dataframe

toronto_grouped.shape

(95, 267)

In [16]:
# Printing each neighbourhood along with top 3 most common venues

num_top_venues = 3

for neighborhood in toronto_grouped['TORNeighborhood']:
    print("----"+neighborhood+"----")
    temp = toronto_grouped[toronto_grouped['TORNeighborhood'] == neighborhood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.18
1            Pool  0.09
2  Sandwich Place  0.09


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0         Bank  0.11
1  Coffee Shop  0.11
2  Pizza Place  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  0.09
2         Coffee Shop  0.09


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2          Café  0.04


----Birch Cliff, Cliffside West----
                   venue  freq
0        College Stadium  0.25
1           Skating Rink  0.25
2  General Entertainment  0.25



In [17]:
# Defining function to sort venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
# Creating new dataframe and displaying top 5 venues for each neighbourhood

import numpy as np

num_top_venues = 5
indicators = ['st', 'nd', 'rd']
columns = ['TORNeighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['TORNeighborhood'] = toronto_grouped['TORNeighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,TORNeighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Clothing Store,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Pub,Pharmacy,Skating Rink,Athletics & Sports
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Pharmacy,Middle Eastern Restaurant
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Department Store
4,"Bedford Park, Lawrence Manor East",Restaurant,Italian Restaurant,Coffee Shop,Sandwich Place,Liquor Store


### 3.3: Cluster Neighbourhoods

In [19]:
# Running k-means to cluster neighbourhoods into 5 clusters

from sklearn.cluster import KMeans

kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('TORNeighborhood', axis=1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 2, 0, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 0], dtype=int32)

In [20]:
# Creating a new dataframe that includes clusters as well as top 5 venues for each neighborhood

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df
toronto_merged.rename(columns={'Neighborhood':'TORNeighborhood'}, inplace=True)
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('TORNeighborhood'), on='TORNeighborhood')

toronto_merged.dropna(axis=0, inplace=True)
toronto_merged = toronto_merged.astype({'Cluster Labels':int})
toronto_merged = toronto_merged.reset_index(drop=True)

toronto_merged.head()

,PostalCode,Borough,TORNeighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2,Fast Food Restaurant,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1,Bar,History Museum,Yoga Studio,Doner Restaurant,Dim Sum Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Rental Car Location,Electronics Store,Breakfast Spot,Medical Center,Intersection
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Soccer Field,Korean Restaurant,Doner Restaurant,Dim Sum Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Caribbean Restaurant


In [21]:
# Visualising resulting clusters

# !conda install -c conda-forge folium=0.5.0 --yes
# Note: Commented above line since package was installed after first run and hence not required to be rerun

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['TORNeighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.4: Examine Clusters

#### Cluster 1

This cluster seems to have a high concentration of outdoor activity areas such as parks, trails, rivers, playgrounds, fields, etc. It can be named as "Neighbourhoods with Many Outdoor Activity Areas".

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,TORNeighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,"Milliken, Agincourt North, Steeles East, L'Amo...",Park,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant
19,"York Mills, Silver Hills",Park,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store
21,York Mills West,Park,Bank,Convenience Store,Ethiopian Restaurant,Electronics Store
38,East Toronto,Park,Convenience Store,Coffee Shop,Dessert Shop,Dim Sum Restaurant
42,Lawrence Park,Park,Bus Line,Swim School,Dog Run,Dim Sum Restaurant
48,Rosedale,Park,Trail,Playground,Department Store,Dessert Shop
71,Humewood-Cedarvale,Trail,Park,Field,Hockey Arena,Yoga Studio
72,Caledonia-Fairbanks,Park,Pool,Women's Store,Colombian Restaurant,Department Store
88,"The Kingsway, Montgomery Road, Old Mill North",River,Park,College Stadium,Colombian Restaurant,Event Space
95,Weston,Park,Convenience Store,Event Space,Ethiopian Restaurant,Electronics Store


#### Cluster 2

This cluster seems to have a high concentration of stores selling daily amenities, food and drinks. It can be named as "Neighbourhoods with Many Stores Offering Essential Services".

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,TORNeighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,"Rouge Hill, Port Union, Highland Creek",Bar,History Museum,Yoga Studio,Doner Restaurant,Dim Sum Restaurant
2,"Guildwood, Morningside, West Hill",Rental Car Location,Electronics Store,Breakfast Spot,Medical Center,Intersection
3,Woburn,Coffee Shop,Soccer Field,Korean Restaurant,Doner Restaurant,Dim Sum Restaurant
4,Cedarbrae,Bakery,Hakka Restaurant,Bank,Athletics & Sports,Caribbean Restaurant
5,Scarborough Village,Convenience Store,Playground,Yoga Studio,Dessert Shop,Dim Sum Restaurant
6,"Kennedy Park, Ionview, East Birchmount Park",Discount Store,Train Station,Convenience Store,Chinese Restaurant,Department Store
7,"Golden Mile, Clairlea, Oakridge",Bus Line,Bakery,Ice Cream Shop,Intersection,Bus Station
8,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Motel,Yoga Studio,Dessert Shop,Dim Sum Restaurant
9,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,Café,General Entertainment,Electronics Store
10,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Chinese Restaurant,Pet Store,Vietnamese Restaurant,Yoga Studio


#### Cluster 3

This cluster seems to be an outlier, possibly due to an uncommon mix of venues as compared to the earlier clusters.

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,TORNeighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Malvern, Rouge",Fast Food Restaurant,Department Store,Falafel Restaurant,Event Space,Ethiopian Restaurant
23,Parkwoods,Park,Fast Food Restaurant,Food & Drink Shop,Dog Run,Dessert Shop


#### Cluster 4

This cluster seems to be an outlier, possibly due to an uncommon mix of venues as compared to the earlier clusters.

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,TORNeighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
77,"North Park, Maple Leaf Park, Upwood Park",Basketball Court,Park,Bakery,Construction & Landscaping,Donut Shop
89,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Construction & Landscaping,Baseball Field,Yoga Studio,Donut Shop,Diner


#### Cluster 5

This cluster seems to be an outlier, possibly due to an uncommon mix of venues as compared to the earlier clusters.

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,TORNeighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
94,"Humberlea, Emery",Baseball Field,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store
